In [125]:
import os
import json
import random
import numpy as np
import tensorflow as tf

In [126]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the senses to be used as inputs for training
x_train_sense = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the sense to be used as inputs for testing
x_test_sense = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [127]:
# Name of directory for grids
directory_name = './data/big_3/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    try:
        data = json.load(f)
    except Exception:
        continue

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        if len(i['gridworld']) != 100:
            print(file_name)
            continue
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_sense.append(i['sense'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()
    

In [128]:
print(direct_count)

[759, 2949, 11111, 14882]


In [129]:
# Name of directory for grids
directory_name = './data/big_3test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_sense.append(i['sense'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [130]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 10, 10) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 10, 10) )
train_in_sense = np.reshape( x_train_sense, (-1, 10, 10) ) / 8
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 10, 10) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 10, 10) )
test_in_sense = np.reshape( x_test_sense, (-1, 10, 10) ) / 8
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [131]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (10,10) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [132]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (10,10) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [133]:
# Flatten the sense input
sense_input = tf.keras.layers.Input( shape = (10,10) )
flatten_sense = tf.keras.layers.Flatten()( sense_input )

In [134]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [135]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position, flatten_sense, flatten_local])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 325), dtype=tf.float32, name=None), name='concatenate_7/concat:0', description="created by layer 'concatenate_7'")


In [136]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 32, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( dense_2 )
dense_4 = tf.keras.layers.Dense( units = 8, activation = tf.nn.relu )( dense_3 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )
probabilities = tf.keras.layers.Softmax()( logits )

In [137]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input, sense_input, local_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [138]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [139]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals], y_test )

11	39	9	0
57	164	76	8
285	388	289	138
163	613	688	0


In [140]:
# Train the model
history = model.fit( [train_in_grid, train_in_position, train_in_sense, train_in_locals], train_out, validation_data=([test_in_grid, test_in_position, test_in_sense, test_in_locals], test_out), epochs = 20 )

Epoch 1/20
929/929 [==============================] - 4s 3ms/step - loss: 0.4424 - accuracy: 0.8525 - val_loss: 0.2883 - val_accuracy: 0.9016
Epoch 2/20
929/929 [==============================] - 2s 3ms/step - loss: 0.2700 - accuracy: 0.9063 - val_loss: 0.2899 - val_accuracy: 0.9071
Epoch 3/20
929/929 [==============================] - 3s 3ms/step - loss: 0.2329 - accuracy: 0.9178 - val_loss: 0.1986 - val_accuracy: 0.9372
Epoch 4/20
929/929 [==============================] - 3s 3ms/step - loss: 0.2082 - accuracy: 0.9291 - val_loss: 0.1831 - val_accuracy: 0.9467
Epoch 5/20
929/929 [==============================] - 2s 3ms/step - loss: 0.1885 - accuracy: 0.9369 - val_loss: 0.1656 - val_accuracy: 0.9498
Epoch 6/20
929/929 [==============================] - 2s 3ms/step - loss: 0.1758 - accuracy: 0.9404 - val_loss: 0.1597 - val_accuracy: 0.9522
Epoch 7/20
929/929 [==============================] - 2s 2ms/step - loss: 0.1655 - accuracy: 0.9450 - val_loss: 0.1553 - val_accuracy: 0.9505
Epoch 

In [141]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals], y_test )

20	26	6	7
3	255	41	6
0	9	1080	11
0	1	8	1455


In [142]:
model.save('./models/agent3_NN')

INFO:tensorflow:Assets written to: ./models/agent3_NN/assets
